In [10]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
from keras.layers import GlobalAveragePooling2D
import gc

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large
# Import modules
import tensorflow as tf
import tensorflow.keras as K


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, BatchNormalization

import numpy as np
import librosa.display, os

# import keras.utils as image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


In [ ]:
N_MELS = 128

In [ ]:
X = np.load('../SavedFeatures/X_mel_spec.npy')
y = np.load('../SavedFeatures/y_mel_spec.npy')

X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [ ]:
def model_vgg16(input_shape, num_classes):
    base_model = VGG16(weights=None, include_top=False, input_shape=input_shape)
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.5)(x)
    
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)


In [26]:
mvgg16 = model_vgg16(input_shape=(N_MELS, 109, 1), num_classes=2)
mvgg16.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 109, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 109, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 109, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 54, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 54, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 27, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 27, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 27, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 27, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 13, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 13, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 13, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,32

 Total params: 15,150,850 (57.80 MB)

 Trainable params: 15,149,826 (57.79 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
mvgg16.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

In [ ]:

tf.keras.backend.clear_session()

In [ ]:

hist = mvgg16.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 36s 49ms/step - accuracy: 0.6838 - loss: 0.6053 - val_accuracy: 0.5936 - val_loss: 0.7187
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.8822 - loss: 0.3027 - val_accuracy: 0.4013 - val_loss: 4.0767
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9353 - loss: 0.1863 - val_accuracy: 0.6828 - val_loss: 0.7068
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9545 - loss: 0.1309 - val_accuracy: 0.4013 - val_loss: 12.7715
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9641 - loss: 0.1178 - val_accuracy: 0.7758 - val_loss: 0.5765
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9678 - loss: 0.0895 - val_accuracy: 0.9686 - val_loss: 0.0829
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9703 - loss: 0.0826 - val_accuracy: 0.8732 - val_loss: 0.5660
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9742 - loss: 0.0762 -

#### Evaluation Metrics

In [30]:
test_loss, test_accuracy = mvgg16.evaluate(X_val, y_val, verbose=0)
y_pred = mvgg16.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step


In [32]:
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 

In [ ]:
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================VGG16 - MS===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")
eers = []

# Tính EER 
for i in range(y_pred.shape[1]):  
    
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


===================VGG16 - MS===================
TAccuracy: 0.82803
F1-Score: 0.82879
Class 0: EER = 0.00489 at threshold 0.00008
Class 1: EER = 0.00525 at threshold 0.99993
EER: 0.00507


In [ ]:
# Save model as weights in H5 format
mvgg16.save_weights("../SavedModel/MS_mvgg16.weights.h5")

In [ ]:

del mvgg16, X_train, X_val, y_train, y_val 
gc.collect()

1043